# Moderate Text in images using Amazon Rekognition

***
This notebook provides a walkthrough of the [text detection API](https://docs.aws.amazon.com/rekognition/latest/dg/text-detection.html) in Amazon Rekognition. You can quickly identify text in your video and image libraries to catalog footage and photos for marketing, advertising, and media industry use cases.
***

# Initialize stuff

In [ ]:
# Initialise Notebook
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [ ]:
# Curent AWS Region. Use this to choose corresponding S3 bucket with sample content

mySession = boto3.session.Session()
awsRegion = mySession.region_name

In [ ]:
# Init clients
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
# S3 bucket that contains sample images and videos

# We are providing sample images and videos in this bucket so
# you do not have to manually download/upload test images and videos.
bucketName = "aws-workshops-" + awsRegion

In [ ]:
# Create temporary directory
# This directory is not needed to call Rekognition APIs.
# We will only use this directory to download images from S3 bucket and drwaw bounding boxes
# around recognized celebrities to show them here in the notebook.

!mkdir m1tmp
tempFolder = 'm1tmp/'

# Detect text in image
***

In [ ]:
imageName = "content-moderation/media/yoga-swimwear.jpg"

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageName})))

#### Call Rekognition to detect text in the image

In [ ]:
# Call Amazon Rekognition to detect text in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectText.html

detectTextResponse = rekognition.detect_text(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Text API (Text Detection)
# In the JSON response below, you will see detected text, confidence score, and additional information.

display(detectTextResponse)

#### Display list of detected unsafe text

In [ ]:
import string
unsafeWords = ["crap", "darn", "damm"]
for textDetection in detectTextResponse["TextDetections"]:
    # strip punctuation before checking match
    text = textDetection["DetectedText"].translate(str.maketrans('', '', string.punctuation))
    if(textDetection["Type"] == "WORD" and text in unsafeWords):
        print("Detected unsafe word: {}".format(textDetection["DetectedText"]))

#### Show image with bounding boxes around detected objects

In [ ]:
# Define a function that will display image with bounded boxes around recognized text
# We will call this function in next step
  
def drawBoundingBoxes (sourceImage, boxes):
    # blue, green, red, grey
    colors = ((255,255,255),(255,255,255),(76,182,252),(52,194,123))
    
    # Download image locally
    imageLocation = tempFolder+os.path.basename(sourceImage)
    s3.download_file(bucketName, sourceImage, imageLocation)

    # Draws BB on Image
    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line= 3
    for box in boxes:
        x1 = int(box[1]['Left'] * width)
        y1 = int(box[1]['Top'] * height)
        x2 = int(box[1]['Left'] * width + box[1]['Width'] * width)
        y2 = int(box[1]['Top'] * height + box[1]['Height']  * height)
        
        draw.text((x1,y1),box[0],colors[col])
        for l in range(line):
            draw.rectangle((x1-l,y1-l,x2+l,y2+l),outline=colors[col])
        col = (col+1)%maxcol
    
    imageFormat = "PNG"
    ext = sourceImage.lower()
    if(ext.endswith('jpg') or ext.endswith('jpeg')):
        imageFormat = 'JPEG'

    bbImage.save(imageLocation,format=imageFormat)

    display(bbImage)

In [ ]:
# Extract bounding box information from JSON response above and display image with bounding boxes around text.

boxes = []
textDetections = detectTextResponse['TextDetections']
for textDetection in textDetections:
        boxes.append ((textDetection['Type'], textDetection["Geometry"]['BoundingBox']))
    
drawBoundingBoxes(imageName, boxes)

***
### References
- https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectText.html

***

You have successfully used Amazon Rekognition to identify text in images an videos.